### Imports

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

### 1. Optimization of a quadratic function

#### Problem definition

$$
\begin{align}
f(x) &= \frac{1}{2} x^T A x\\
A &= diag(1 \dots 1000)
\end{align}
$$

Our starting point is given to be: $x_0 = [1000, 999, \dots, 1]^T$

We are constrained to use steepest descent and to terminate when we satisfy the following condition:

$$
||\nabla f(x_k)||_2 \leq 10^{-6} ||\nabla f(x_0)||_2
$$

Our goal is to minimize the number of function evalutions $f(x_k)$ by optimizing the step size $\alpha_k$.

In [2]:
# define the constants
A = np.diag(np.arange(1, 1001))
x0 = np.arange(1000, 0, -1)
print("x0 shape: ", x0.shape)
print("A shape: ", A.shape) 

# define the function
def Q(x):
    return 0.5 * x.T @ A @ x
print("f(x0) shape: ", Q(x0).shape)

# define the gradient
def dQ(x):
    return A @ x
print("df(x0) shape: ", dQ(x0).shape)

# compute 2 norm
def norm(x):
    return np.sqrt(x.T @ x)

tol = 1e-6 * norm(dQ(x0))
print("tol: ", tol)

x0 shape:  (1000,)
A shape:  (1000, 1000)
f(x0) shape:  ()
df(x0) shape:  (1000,)
tol:  5.787947275744656



#### Steepest descent

The general idea behind iterative methods is to compute a step along search direction $d_k$ and update the current point $x_k$ by adding the step $d_k$ to it. The history of steepest descent is very well explained in [1].

$$
\begin{align}
x_{k+1} &= x_k + \alpha_k d_k\\
\end{align}
$$

For the method of steepest descent the direction is given by the negative gradient of the function $f$:

$$
\begin{align}
d_k &= -\nabla f(x_k)\\
x_{k+1} &= x_k - \alpha_k \nabla f(x_k)\\
\end{align}
$$

For our quadratic model $f(x) = \frac{1}{2} x^T A x$ we have:

$$
\begin{align}
\nabla f(x) &= Ax\\
&= diag(1 \dots 1000) x
\end{align}
$$

So that:

$$
\begin{align}
x_{k+1} &= x_k - \alpha_k A x_k\\
\end{align}
$$

#### Step size optimization

We can optimize the step size $\alpha_k$ by solving the following optimization problem, which only makes sense for convex $f$:

$$
\begin{align}
\alpha_k &= \arg\min_{\alpha} f(x_k - \alpha \nabla f(x_k))\\
         &= \arg\min_{\alpha} f(x_k - \alpha A x_k)\\
         &= \arg\min_{\alpha} \frac{1}{2} (x_k - \alpha A x_k)^T A (x_k - \alpha A x_k)\\
\end{align}
$$

So that:

$$
\begin{align}
\alpha_k &= \frac{d_k^T d_k}{d_k^T A d_k}\\
         &= \frac{x_k^T A^2 x_k}{x_k^T A^3 x_k}\\
\end{align}
$$




[1] Meza, J. C. (2010). Steepest descent. Wiley Interdisciplinary Reviews: Computational Statistics, 2(6), 719-722.

#### Naive implementation

We take the most simple solution, where $\alpha_k = c > 0$ is a small constant. 

In [3]:
from typing import Callable
from IPython.display import clear_output, display
import matplotlib.pyplot as plt

class Optim:
    """Generic implementation of an optimization algorithm class."""
    def __init__(self, x0: np.ndarray, f: Callable, new_alpha: Callable, df: Callable, tol: float):
        """
        :param x0: initial guess
        :param f: objective function
        :param new_alpha: update rule to obtain new alpha_k
        :param df: gradient of f
        :param tol: tolerance for stopping criterion
        """
    
        self.x = x0
        self.f = f
        self.new_alpha = new_alpha
        self.df = df
        self.tol = tol
        self.fs = [f(x0)]

    def optimize(self):
        """Optimize the objective function."""
        # starting point
        x_k = self.x
        alpha_k = self.new_alpha(x_k)

        # iterate until stopping criterion is satisfied
        idx = 0
        norm_df_xk = np.inf

        while norm_df_xk > self.tol:
            clear_output(wait=True)

            # update x_k and alpha_k
            x_k = x_k - alpha_k * self.df(x_k)
            alpha_k = self.new_alpha(x_k)

            # store the function value
            f_xk = self.f(x_k)
            norm_df_xk = norm(self.df(x_k))
            self.fs.append(f_xk)

            # print the iteration
            display(f"Iteration {idx}: f(x) = {f_xk:.2f}, ||df(x)|| = {norm_df_xk:.2f}")
            idx += 1

        self.x = x_k

In [4]:
naive_optim = Optim(x0, Q, lambda x: 1e-3, dQ, tol)
naive_optim.optimize()
print(f"Number of function evaluations: {len(naive_optim.fs)}")

'Iteration 5149: f(x) = 16.73, ||df(x)|| = 5.78'

Number of function evaluations: 5151


#### Optimal implementation, see Eq. (2)

In [5]:
optimal_alpha_k = lambda x: (x.T @ A @ A @ x) / (x.T @ A @ A @ A @ x)
optimal_optim = Optim(x0, Q, optimal_alpha_k, dQ, tol)
optimal_optim.optimize()
print(f"Number of function evaluations: {len(optimal_optim.fs)}")

'Iteration 2749: f(x) = 8.36, ||df(x)|| = 5.78'

Number of function evaluations: 2751


#### Line search with Armijo condition

The Armijo condition is given by, see [2] Eq. 3.4:

$$
\begin{align}
f(x_k + \alpha_k d_k) &\leq f(x_k) + c_1 \alpha_k \nabla f(x_k)^T d_k\\
\end{align}
$$

Where $c_1 \in (0, 1)$ is a small constant. We can use the Armijo condition to find a step size $\alpha_k$ that satisfies the condition. We start with $\alpha_k = 1$ and decrease it by a factor of $0.5$ until the condition is satisfied.

[2] Jorge, N., & Stephen, J. W. (2006). Numerical optimization. Spinger.

In [12]:
# define update by Armijo rule
def armijo_alpha_k(x_k, alpha=1, beta=0.5, c_1=0.01):
    dQ_xk = dQ(x_k)
    dQ_xk_T = np.transpose(dQ_xk)
    while Q(x_k - alpha * dQ_xk) > Q(x_k) - c_1 * alpha * dQ_xk_T @ dQ_xk:
        alpha *= beta
    return alpha

armijo_optim = Optim(x0, Q, armijo_alpha_k, dQ, tol)
armijo_optim.optimize()
print(f"Number of function evaluations: {len(armijo_optim.fs)}")

'Iteration 2714: f(x) = 7.92, ||df(x)|| = 5.76'

Number of function evaluations: 2716


#### Line search with Armijo + curvature conditions (Wolfe conditions)

The Wolfe conditions are given by, see [2] Eq. 3.6a and 3.6b:

$$
\begin{align}
f(x_k + \alpha_k d_k) &\leq f(x_k) + c_1 \alpha_k \nabla f(x_k)^T d_k\\
\nabla f(x_k + \alpha_k d_k)^T d_k &\geq c_2 \nabla f(x_k)^T d_k\\
\end{align}
$$

Where $c_1 \in (0, 1)$ and $c_2 \in (c_1, 1)$ are small constants. We can use the Wolfe conditions to find a step size $\alpha_k$ that satisfies the conditions. We start with $\alpha_k = 1$ and decrease it by a factor of $0.5$ until the conditions are satisfied.

In [18]:
# define update rule with Wolfe conditions
def wolfe_alpha_k(x_k, alpha=1, beta=0.5, c_1=0.5, c_2=0.9):
    dQ_xk = dQ(x_k)
    dQ_xk_T = np.transpose(dQ_xk)
    while Q(x_k - alpha * dQ_xk) > Q(x_k) - c_1 * alpha * dQ_xk_T @ dQ_xk and \
          dQ(x_k - alpha * dQ_xk) @ dQ_xk < c_2 * dQ_xk_T @ dQ_xk:
        alpha *= beta
    return alpha

wolfe_optim = Optim(x0, Q, wolfe_alpha_k, dQ, tol)
wolfe_optim.optimize()
print(f"Number of function evaluations: {len(wolfe_optim.fs)}")

'Iteration 1291: f(x) = 14.76, ||df(x)|| = 5.62'

Number of function evaluations: 1293


#### Barzilai and Borwein method

The Barzilai and Borwein method is given by, see [1] Eq. 13 and 14:

We write the iterate update rule as:

$$
\begin{align}
x_{k+1} &= x_k - \frac{1}{\alpha_k}  d_k\\
\end{align}
$$

We then compute the step length according to the formula:

$$
\begin{align}
\alpha_k &= \frac{s_{k-1}^T y_{k-1}}{s_{k-1}^T s0_{k-1}}\\
\end{align}
$$

Where $s_{k-1} = x_k - x_{k-1}$ and $y_{k-1} = d_k - d_{k-1}$.

[1] Barzilai, J., & Borwein, J. M. (1988). Two-point step size gradient methods. IMA journal of numerical analysis, 8(1), 141-148.

In [ ]:
# barzilai and borwein method
def barzilai_borwein_alpha_k(x_k, alpha=1):